This notebook runs a pipeline with the following steps:
* Create a test Visibility dataset with two components and no beam.
* Write to disk in MSv2 format.
* Read MSv2 data into Visibility dataset.$^{*}$
* Predict model visibilities.$^{*}$
* Do bandpass calibration.$^{*}$
* Check for convergence.$^{*}$

\* xarray dataset map_blocks() is used to distribute frequency sub-bands across dask tasks.

The pipeline is called once with the default internal dask cluster,
then again with an externally generated cluster.

In [1]:
from time import perf_counter

from dask.distributed import LocalCluster
from ska_sdp_instrumental_calibration.processing_tasks.lsm import Component
from ska_sdp_instrumental_calibration.workflow.pipelines import (
    bandpass_calibration,
)

<frozen abc>:106: FutureWarning: xarray subclass Visibility should explicitly define __slots__


In [2]:
# Simple bandpass calibration with internally generated cluster
t0 = perf_counter()
bandpass_calibration.run(
    {
        "lsm": [
            Component(
                name="comp1", RAdeg=20.0, DEdeg=-30.0, flux=1.0, alpha=0.0
            ),
            Component(
                name="comp2", RAdeg=19.5, DEdeg=-29.9, flux=0.1, alpha=0.0
            ),
        ],
        "beam_type": "none",
        "ms_name": "demo.ms",
        "hdf5_name": "demo.hdf5",
    }
)
print(f"pipeline ran in {perf_counter() - t0:.1f} seconds")

2025-02-05 14:58:09,866 - workflow.pipeline_config - INFO - Simulating demo MSv2 input
2025-02-05 14:58:09,868 - workflow.pipeline_config - INFO - Predicting visibilities without a beam
2025-02-05 14:58:09,868 - workflow.pipeline_config - INFO - Generating a demo MSv2 Measurement Set demo.ms
2025-02-05 14:58:09,880 - workflow.utils - INFO - Using AA1-Low with 20 stations
2025-02-05 14:58:10,422 - processing_tasks.predict - INFO - No beam model used in predict
2025-02-05 14:58:10,578 - workflow.utils - INFO - Applying direction-independent gain corruptions
2025-02-05 14:58:11,172 - pipeline.bandpass_calibration - INFO - Starting pipeline with 16-channel chunks
2025-02-05 14:58:11,173 - pipeline.bandpass_calibration - INFO - No dask cluster supplied. Using LocalCluster
2025-02-05 14:58:12,395 - pipeline.bandpass_calibration - INFO - Setting calibration in 16-channel chunks
2025-02-05 14:58:12,396 - pipeline.bandpass_calibration - INFO - end_to_end_subbands = true
2025-02-05 14:58:12,631 

pipeline ran in 5.4 seconds


In [3]:
# Simple bandpass calibration with externally generated cluster
t0 = perf_counter()
cluster = LocalCluster()
print(f"Sending scheduler address {cluster.scheduler_address}")
bandpass_calibration.run(
    {
        "dask_scheduler_address": cluster.scheduler_address,
        "lsm": [
            Component(
                name="comp1", RAdeg=20.0, DEdeg=-30.0, flux=1.0, alpha=0.0
            ),
            Component(
                name="comp2", RAdeg=19.5, DEdeg=-29.9, flux=0.1, alpha=0.0
            ),
        ],
        "beam_type": "none",
        "ms_name": "demo.ms",
        "hdf5_name": "demo.hdf5",
    }
)
print(f"pipeline ran in {perf_counter() - t0:.1f} seconds")

2025-02-05 14:58:16,046 - workflow.pipeline_config - INFO - Simulating demo MSv2 input
2025-02-05 14:58:16,047 - workflow.pipeline_config - INFO - Predicting visibilities without a beam
2025-02-05 14:58:16,048 - workflow.pipeline_config - INFO - Generating a demo MSv2 Measurement Set demo.ms
2025-02-05 14:58:16,072 - workflow.utils - INFO - Using AA1-Low with 20 stations
2025-02-05 14:58:16,123 - processing_tasks.predict - INFO - No beam model used in predict


Sending scheduler address tcp://127.0.0.1:33913


2025-02-05 14:58:16,248 - workflow.utils - INFO - Applying direction-independent gain corruptions
2025-02-05 14:58:16,844 - pipeline.bandpass_calibration - INFO - Starting pipeline with 16-channel chunks
2025-02-05 14:58:16,845 - pipeline.bandpass_calibration - INFO - Using existing dask cluster tcp://127.0.0.1:33913
2025-02-05 14:58:16,858 - pipeline.bandpass_calibration - INFO - Setting calibration in 16-channel chunks
2025-02-05 14:58:16,859 - pipeline.bandpass_calibration - INFO - end_to_end_subbands = true
2025-02-05 14:58:16,936 - pipeline.bandpass_calibration - INFO - Running graph and returning calibration solutions
2025-02-05 14:58:18,111 - processing_tasks.predict - INFO - No beam model used in predict
2025-02-05 14:58:18,190 - processing_tasks.predict - INFO - No beam model used in predict
2025-02-05 14:58:18,993 - processing_tasks.predict - INFO - No beam model used in predict
2025-02-05 14:58:19,027 - processing_tasks.predict - INFO - No beam model used in predict
2025-02-

pipeline ran in 4.0 seconds
